# Setup

## Install necessary libraries

In [ ]:
! pip install langchain_community tiktoken langchainhub langchain langchain-google-genai
! pip install -qU langchain-huggingface sentence_transformers chromadb  langchain-qdrant qdrant_client

## Insert API key


You will need first to get an API key from [Google AI Studio](https://aistudio.google.com/app/apikey).

In [ ]:

import os

GOOGLE_API_KEY = ""  # add your GOOGLE API key here
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

#or run this
#from google.colab import userdata
#from google.colab import drive
#os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Indexing


## 1. Load Data

### From your files

In [ ]:
!pip install unstructured
!pip install "unstructured[pdf]"

In [ ]:
import os
from google.colab import files
import shutil

# Create the folder
folder_path = os.path.join("/content/", "uploaded_files")
os.makedirs(folder_path, exist_ok=True)  # Create if it doesn't exist

# Upload files
uploaded = files.upload()

# Move uploaded files to the folder
for filename, data in uploaded.items():
  source_path = os.path.join("/content/", filename)  # Path to uploaded file
  destination_path = os.path.join(folder_path, filename)
  shutil.move(source_path, destination_path)  # Move the file

In [ ]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(folder_path)
docs = loader.load()
len(docs)

2

### From URLs

In [ ]:
import bs4

from langchain_community.document_loaders import WebBaseLoader

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
len(docs)

2

## 2. Use a Text Splitter to Split Documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

241

## 3. Embed the documents and store them

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
#sentence-transformers/all-MiniLM-L6-v2
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity

)
print(f"Model's maximum sequence length: {SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2').max_seq_length}")

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(
    path="/content/vector_store_folder"
    #":memory:"
    # you can use :memory: mode for fast and light-weight experiments,
)
client.create_collection(
    collection_name="workshop_collection",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="workshop_collection",
    embedding=embedding_model
)
vector_store.add_documents(documents=splits)


### In case qdrant doesnt work, use this:

In [ ]:
vector_store = Chroma.from_documents(documents=splits,
                                    embedding=embedding_model)

# Retrieval

In [ ]:
user_query= "What are the three core parts of an agent?" #"Which system became the first AI to earn an IMO medal?"

In [ ]:
retrieved_docs = vector_store.similarity_search_with_score(query=user_query, k=5)

In [ ]:
retrieved_docs

[(Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': '2ba4a80cad7c4ddeaafab4d6122e52c4', '_collection_name': 'workshop_collection'}, page_content='Relationships between agents and observations of one agent by another are all taken into consideration for planning and reacting.\nEnvironment information is present in a tree structure.'),
  0.5085810586457418),
 (Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': '3d05827017bc439abe5156624b04d63b', '_collection_name': 'workshop_collection'}, page_content='Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.'),
  0.4987262613833804),
 (Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': '9a30f9c352af44eeade5b1b372ad651c', '_collection_name': 'workshop_collection'}, page_content='Fig. 13. The generative agent architecture. (Image sour

In [ ]:
print("Content:",retrieved_docs[0][0].page_content)
print("Metadata",retrieved_docs[0][0].metadata)
print("Similarity score",retrieved_docs[0][1])

Content: Relationships between agents and observations of one agent by another are all taken into consideration for planning and reacting.
Environment information is present in a tree structure.
Metadata {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', '_id': '2ba4a80cad7c4ddeaafab4d6122e52c4', '_collection_name': 'workshop_collection'}
Similarity score 0.5085810586457418


## Create Retriever

In [ ]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})
docs = retriever.get_relevant_documents(user_query)
print(docs[0].page_content)

<ipython-input-28-c35d213439dc>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(user_query)


Relationships between agents and observations of one agent by another are all taken into consideration for planning and reacting.
Environment information is present in a tree structure.


# Generation

## Define prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate


# Prompt

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.If the provided context doesn't contain the answer, answer from your knowledge but say that you do,else just say that you don't know, don't try to make up an answer.
Question: {question}
Context: {context}
"""

prompt = ChatPromptTemplate.from_template(template)

## Configure LLM

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",#"gemini-2.0-flash",#"gemini-2.5-flash-preview-04-17",#
    temperature=0,
    # max_tokens=None,
    # timeout=None,
    # max_retries=2,
)

## Create chain and invoke

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
response=rag_chain.invoke(user_query)

In [ ]:
print(response)

I'm sorry, but the provided text does not explicitly state the three core parts of an agent.


# Evaluating the pipeline

In [ ]:
!pip install ragas

In [ ]:
from ragas import EvaluationDataset

#sample queries and expected responses for the sample pdf files given
# sample_queries = ["What are the three core parts of an agent?",
#              "Which hypothesis says a single reward can be enough for intelligence?",
#              "Which system became the first AI to earn an IMO medal?",
#              "What are the two basic multi-agent orchestration patterns?",
#              "List the three steps in the guardrail-setup heuristic."
#             ]

# #If you are not interested in the context_recall metric, you don’t need to provide the ground_truths information.
# expected_responses = ["Model, Tools, Instructions",
#                 "'Reward is Enough' hypothesis",
#                 "AlphaProof",
#                 "Manager pattern and Decentralised pattern",
#                 "1 Focus on privacy & safety 2 Add guardrails for real-world edge cases 3 Tune for both security and user experience"
#                  ]

# sample queries and expected responses for the sample website given
sample_queries = ["What three components sit alongside the LLM “brain” in an autonomous agent system?",
             "What are the two main kinds of memory an agent maintains?",
             "Which prompting method tells the model to 'think step by step'?",
             "Which framework fuses reasoning traces with discrete actions inside an agent?",
             "Generative Agents score memories on three factors; name them."
            ]

#If you are not interested in the context_recall metric, you don’t need to provide the ground_truths information.
expected_responses = ["Planning, Memory, Tool use",
                "Short-term memory and Long-term memory",
                "Chain of Thought (CoT)",
                "ReAct",
                "Recency, Importance, Relevance"
                 ]

dataset = []

for query, reference in zip(sample_queries, expected_responses):
    relevant_docs = retriever.invoke(query)
    response = rag_chain.invoke(query)
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

evaluator_llm = LangchainLLMWrapper(llm)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
)



In [ ]:
print(result)


{'context_recall': 0.2667, 'faithfulness': 0.5139, 'factual_correctness(mode=f1)': 0.4480}
